In [1]:
!nvidia-smi

Wed Feb 12 18:59:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whic

In [3]:
from datasets import load_dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

NUM_EPOCHS = 50
PATIENCE = 3

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:
def load_mnist_dataset():
    mnist_dataset = load_dataset("mnist")
    transform = transforms.ToTensor()

    def _transform_example(example):
        example["image"] = transform(example["image"])
        return example

    mnist_dataset = mnist_dataset.map(_transform_example)
    mnist_dataset.set_format(type="torch", columns=["image", "label"])

    train_dataset = mnist_dataset["train"]
    test_dataset = mnist_dataset["test"]

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

    return train_loader, test_loader

In [6]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dim=64):
        super(VariationalAutoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.dropout = nn.Dropout(0.2)

        self.encoder_fc1 = nn.Linear(28 * 28, 128)
        self.encoder_bn1 = nn.BatchNorm1d(128)
        self.encoder_fc2 = nn.Linear(128, 64)
        self.encoder_bn2 = nn.BatchNorm1d(64)

        self.fc_mean = nn.Linear(64, latent_dim)
        self.fc_logvar = nn.Linear(64, latent_dim)

        self.decoder_fc1 = nn.Linear(latent_dim, 64)
        self.decoder_bn1 = nn.BatchNorm1d(64)
        self.decoder_fc2 = nn.Linear(64, 128)
        self.decoder_bn2 = nn.BatchNorm1d(128)
        self.decoder_fc3 = nn.Linear(128, 28 * 28)

    def encode(self, x):
        x = x.view(x.size(0), -1)  # [Batch, 1, 28, 28] → [Batch, 784]
        h = self.encoder_fc1(x)
        h = self.encoder_bn1(h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.encoder_fc2(h)
        h = self.encoder_bn2(h)
        h = F.relu(h)
        h = self.dropout(h)
        mu = self.fc_mean(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)  # Sample
        return mu + eps * std

    def decode(self, z):
        h = self.decoder_fc1(z)
        h = self.decoder_bn1(h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.decoder_fc2(h)
        h = self.decoder_bn2(h)
        h = F.relu(h)
        h = self.dropout(h)
        out = self.decoder_fc3(h)
        out = torch.sigmoid(out)  # Normalize → [0, 1]
        out = out.view(-1, 1, 28, 28)
        return out

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        decoded = self.decode(z)
        return decoded, mu, logvar

    def get_latent(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return z

# Loss-Funktion für den VAE mit β-VAE und KL-Annealing
def vae_loss(recon_x, x, mu, logvar, beta=1.0):
    """
    beta: Gewichtung der KL-Divergenz (nützlich für β-VAE)
    """
    BCE = F.binary_cross_entropy(recon_x.view(-1, 28*28), x.view(-1, 28*28), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  # KL-Verlust
    return BCE + beta * KLD  # β-KL-Verlust-Kontrolle

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride: int = 1, dropout_prob: float = 0.2):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.dropout = nn.Dropout(dropout_prob)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class ResidualCNN(nn.Module):
    def __init__(self):
        super(ResidualCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = ResidualBlock(16, 16, stride=1, dropout_prob=0.2)
        self.layer2 = ResidualBlock(16, 32, stride=2, dropout_prob=0.2)
        self.layer3 = ResidualBlock(32, 64, stride=2, dropout_prob=0.2)
        self.layer4 = ResidualBlock(64, 64, stride=1, dropout_prob=0.2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout_fc = nn.Dropout(0.5)
        self.fc = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout_fc(x)
        x = self.fc(x)
        return x

In [8]:
class LatentCNN(nn.Module):
    def __init__(self):
        super(LatentCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=0.5)  # Dropout-Wahrscheinlichkeit von 50%
        self.fc = nn.Linear(32 * 8 * 8, 10)

    def forward(self, x):
        x = x.view(-1, 1, 8, 8)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [9]:
# ---------------------------
# Evaluation functions
# ---------------------------
def evaluate_variational_autoencoder(model, data_loader, device, beta):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in data_loader:
            imgs = batch["image"].to(device)
            recon_x, mu, logvar = model(imgs)
            loss = vae_loss(recon_x, imgs, mu, logvar, beta)
            total_loss += loss.item()
    avg_loss = total_loss / len(data_loader.dataset)
    return avg_loss

def evaluate_residualcnn(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in data_loader:
            imgs = batch["image"].to(device)
            labels = batch["label"].to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    avg_loss = total_loss / len(data_loader.dataset)
    accuracy = correct / total
    return avg_loss, accuracy

def evaluate_latentcnn(variational_autoencoder, latentcnn, data_loader, criterion, device):
    variational_autoencoder.eval()
    latentcnn.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in data_loader:
            imgs = batch["image"].to(device)
            labels = batch["label"].to(device)
            latent_imgs = variational_autoencoder.get_latent(imgs)
            outputs = latentcnn(latent_imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    avg_loss = total_loss / len(data_loader.dataset)
    accuracy = correct / total
    return avg_loss, accuracy

In [10]:
train_loader, test_loader = load_mnist_dataset()

variational_autoencoder_model = VariationalAutoencoder().to(device)
latentcnn_model = LatentCNN().to(device)
residualcnn_model = ResidualCNN().to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
# ---------------------------------
# Training Variational Autoencoder mit KL-Annealing
# ---------------------------------

# Hyperparameter für KL-Annealing
beta_start = 0.0
beta_end = 1.0
annealing_epochs = 20  # Nach 20 Epochen erreicht β sein Maximum

# Lossfunktion für den VAE mit β-VAE und KL-Annealing
def vae_loss(recon_x, x, mu, logvar, beta=1.0):
    BCE = F.binary_cross_entropy(recon_x.view(-1, 28*28), x.view(-1, 28*28), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  # KL-Verlust
    return BCE + beta * KLD  # β-KL-Verlust-Kontrolle

print("Training VAE with KL-Annealing")
optimizer_vae = optim.Adam(variational_autoencoder_model.parameters(), lr=1e-3)

best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in range(NUM_EPOCHS):
    variational_autoencoder_model.train()

    # KL-Annealing: β-Wert schrittweise erhöhen
    beta = min(beta_end, beta_start + (epoch / annealing_epochs) * (beta_end - beta_start))

    progress_bar = tqdm(train_loader, desc=f"VAE Epoch {epoch+1}/{NUM_EPOCHS}", unit="batch")
    for batch in progress_bar:
        imgs = batch["image"].to(device)
        recon_x, mu, logvar = variational_autoencoder_model(imgs)
        loss = vae_loss(recon_x, imgs, mu, logvar, beta)

        optimizer_vae.zero_grad()
        loss.backward()
        optimizer_vae.step()

        progress_bar.set_postfix(loss=loss.item(), beta=beta)

    val_loss = evaluate_variational_autoencoder(variational_autoencoder_model, test_loader, device, beta)
    print(f"VAE Epoch {epoch+1} Validation Loss: {val_loss:.4f}, β: {beta:.4f}")

    # Early Stopping Logik
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        # Modell speichern
        # torch.save(variational_autoencoder_model.state_dict(), "best_vae_model.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("Early stopping for VAE")
            break

Training VAE with KL-Annealing


VAE Epoch 1/50: 100%|██████████| 938/938 [00:37<00:00, 25.17batch/s, beta=0, loss=4.64e+3]


VAE Epoch 1 Validation Loss: 119.8815, β: 0.0000


VAE Epoch 2/50: 100%|██████████| 938/938 [00:38<00:00, 24.59batch/s, beta=0.05, loss=4.53e+3]


VAE Epoch 2 Validation Loss: 116.9778, β: 0.0500


...

VAE Epoch 8 Validation Loss: 115.5768, β: 0.3500


VAE Epoch 9/50: 100%|██████████| 938/938 [00:36<00:00, 25.38batch/s, beta=0.4, loss=4.45e+3]


VAE Epoch 9 Validation Loss: 115.6287, β: 0.4000


VAE Epoch 10/50: 100%|██████████| 938/938 [00:36<00:00, 25.58batch/s, beta=0.45, loss=4.61e+3]


VAE Epoch 10 Validation Loss: 116.1421, β: 0.4500
Early stopping for VAE


In [12]:
# ---------------------------------
# Training LatentCNN (Klassifikation über den latenten Raum des Autoencoders)
# ---------------------------------
print("\nTraining LatentCNN")
criterion_cls = nn.CrossEntropyLoss()
optimizer_lat = optim.Adam(latentcnn_model.parameters(), lr=1e-3)

best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in range(NUM_EPOCHS):
    latentcnn_model.train()
    progress_bar = tqdm(train_loader, desc=f"LatentCNN Epoch {epoch+1}/{NUM_EPOCHS}", unit="batch")
    for batch in progress_bar:
        imgs = batch["image"].to(device)
        labels = batch["label"].to(device)
        # Hole die latente Darstellung vom Autoencoder
        latent_imgs = variational_autoencoder_model.get_latent(imgs)
        outputs = latentcnn_model(latent_imgs)
        loss = criterion_cls(outputs, labels)

        optimizer_lat.zero_grad()
        loss.backward()
        optimizer_lat.step()

        progress_bar.set_postfix(loss=loss.item())

    val_loss, val_acc = evaluate_latentcnn(variational_autoencoder_model, latentcnn_model, test_loader, criterion_cls, device)
    print(f"LatentCNN Epoch {epoch+1} Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("Early stopping for LatentCNN")
            break


Training LatentCNN


LatentCNN Epoch 1/50: 100%|██████████| 938/938 [00:36<00:00, 25.38batch/s, loss=0.598]


LatentCNN Epoch 1 Validation Loss: 0.4716, Accuracy: 0.8569


LatentCNN Epoch 2/50: 100%|██████████| 938/938 [00:35<00:00, 26.26batch/s, loss=0.476]


LatentCNN Epoch 2 Validation Loss: 0.4456, Accuracy: 0.8621


...

LatentCNN Epoch 13 Validation Loss: 0.3856, Accuracy: 0.8810


LatentCNN Epoch 14/50: 100%|██████████| 938/938 [00:35<00:00, 26.19batch/s, loss=0.836]


LatentCNN Epoch 14 Validation Loss: 0.3831, Accuracy: 0.8824


LatentCNN Epoch 15/50: 100%|██████████| 938/938 [00:36<00:00, 25.87batch/s, loss=0.305]


LatentCNN Epoch 15 Validation Loss: 0.3832, Accuracy: 0.8832
Early stopping for LatentCNN


In [13]:
# ---------------------------------
# Training ResidualCNN (Klassifikation, CrossEntropyLoss)
# ---------------------------------
print("\nTraining ResidualCNN")
criterion_cls = nn.CrossEntropyLoss()
optimizer_res = optim.Adam(residualcnn_model.parameters(), lr=1e-3)

best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in range(NUM_EPOCHS):
    residualcnn_model.train()
    progress_bar = tqdm(train_loader, desc=f"ResidualCNN Epoch {epoch+1}/{NUM_EPOCHS}", unit="batch")
    for batch in progress_bar:
        imgs = batch["image"].to(device)
        labels = batch["label"].to(device)
        outputs = residualcnn_model(imgs)
        loss = criterion_cls(outputs, labels)

        optimizer_res.zero_grad()
        loss.backward()
        optimizer_res.step()

        progress_bar.set_postfix(loss=loss.item())

    val_loss, val_acc = evaluate_residualcnn(residualcnn_model, test_loader, criterion_cls, device)
    print(f"ResidualCNN Epoch {epoch+1} Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("Early stopping for ResidualCNN")
            break


Training ResidualCNN


ResidualCNN Epoch 1/50: 100%|██████████| 938/938 [00:40<00:00, 22.96batch/s, loss=0.137]


ResidualCNN Epoch 1 Validation Loss: 0.0478, Accuracy: 0.9853


ResidualCNN Epoch 2/50: 100%|██████████| 938/938 [00:40<00:00, 22.89batch/s, loss=0.108]


ResidualCNN Epoch 2 Validation Loss: 0.0565, Accuracy: 0.9816


...

ResidualCNN Epoch 9 Validation Loss: 0.0182, Accuracy: 0.9942


ResidualCNN Epoch 10/50: 100%|██████████| 938/938 [00:40<00:00, 23.33batch/s, loss=0.057]


ResidualCNN Epoch 10 Validation Loss: 0.0203, Accuracy: 0.9939


ResidualCNN Epoch 11/50: 100%|██████████| 938/938 [00:40<00:00, 23.28batch/s, loss=0.00301]


ResidualCNN Epoch 11 Validation Loss: 0.0173, Accuracy: 0.9950
Early stopping for ResidualCNN


In [14]:
# ---------------------------------
# Evaluate LatentCNN
# ---------------------------------

latentcnn_model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating", unit="batch"):
        imgs = batch["image"].to(device)
        labels = batch["label"].to(device)
        x = variational_autoencoder_model.get_latent(imgs)
        outputs = latentcnn_model(x)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Berechnung der Metriken mit scikit-learn
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

Evaluating: 100%|██████████| 157/157 [00:05<00:00, 29.40batch/s]

Test Accuracy: 0.8811
Test Precision: 0.8800
Test Recall: 0.8794
Test F1 Score: 0.8794


In [15]:
# ---------------------------------
# Residual LatentCNN
# ---------------------------------

residualcnn_model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating", unit="batch"):
        imgs = batch["image"].to(device)
        labels = batch["label"].to(device)
        outputs = residualcnn_model(imgs)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Berechnung der Metriken mit scikit-learn
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

Evaluating: 100%|██████████| 157/157 [00:04<00:00, 33.04batch/s]

Test Accuracy: 0.9950
Test Precision: 0.9950
Test Recall: 0.9949
Test F1 Score: 0.9949
